In [6]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(".."))

from typing import Union
from src.preprocessing import compute_cancellation

In [2]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df = pd.read_parquet(root + '/data/train.parquet')
df.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [7]:
sample_time = df['time'].min() if 'time' in df.columns else '2023-01-01'
target_df = compute_cancellation(df, present_time=sample_time)
print('Computed cancellation target for', len(target_df), 'unique users')
print(f'Cancellations within 10 days of {sample_time}:')
print(target_df['cancellation_confirmed'].value_counts())
target_df.head(10)

Computed cancellation target for 19140 unique users
Cancellations within 10 days of 2018-10-01 00:00:01:
cancellation_confirmed
0    18062
1     1078
Name: count, dtype: int64


,userId,cancellation_confirmed
0,1749042,0
1,1563081,0
2,1697168,0
3,1222580,0
4,1714398,0
5,1010522,0
6,1475659,0
7,1558463,0
8,1605667,0
9,1385500,0
